<a href="https://colab.research.google.com/github/caroheymes/Descente-de-gradients/blob/main/01_Linear_regression_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Doing a Linear regression without Scikit Learn

There's nothing better to understand the gradient descent algorithm than to code it from scratch. It may seem difficult at first glance but don't worry, we will guide you step by step. It is also a fantastic occasion to practice python key concepts like **Classes**!

Don't hesitate to come back to your Machine Learning course on linear regression to refresh your memory. 

Our goal will be to code a simple linear regression such as : 

$f(x) = \beta_1 \times x + \beta_0$

* Import the following libraries: 
  * Numpy 

In [1]:
import os
from google.colab import drive
drive.mount('/content/drive', force_remount = True)


Mounted at /content/drive


In [4]:
import numpy as np 

* Define a `Model` class that will take two methods: 
  1. `__init__(self)`, the class builder which will allow you to define an attribute $\beta_0$ (`beta_0` in your code) and an attribute $\beta_1$ (`beta_1` in your code). These attributes represent the coefficients/parameters of the model an we will be initialize them randomly using Numpy (cf: `np.random.randn`).
  2. `__call__(self, x)`, a special method that will turn our class into a callable which will return $\beta_1 \times x + \beta_0$ when called.

In [135]:
class Model():
  def __init__(self):
    self.beta_1 = np.random.randn(1)
    self.beta_0 = np.random.randn(1)
  def __call__(self, x):
    return self.beta_1 * x + self.beta_0

* Create an instance of your class `Model`

In [136]:
model = Model()

In [137]:
model.beta_0

array([0.07262203])

In [138]:
model.beta_1

array([0.05243761])

* Try doing a first "regression" by running `model(3.0)`. 
NB: If you don't have the same values as this notebook in output, this is normal since you have initialized your values randomly. 

In [139]:
model(3)

array([0.22993486])

* This value corresponds to a random prediction of your model. But we don't have any data yet. This time, let's use `sklearn` to import data. 
  * Import `sklearn.datasets`
  * Use the `load_diabetes()` function to load the diebetes dataset in an object called `diabetes`.
  * Print the `DESCR` attribute of the diabetes object
  * Save the content of the `data` attribute in an object named `diabetes_data`
  * Save the content of the `target` attribute in an object named `y`

In [140]:
import sklearn.datasets
diabetes = sklearn.datasets.load_diabetes()

In [141]:
print(diabetes.DESCR)

.. _diabetes_dataset:

Diabetes dataset
----------------

Ten baseline variables, age, sex, body mass index, average blood
pressure, and six blood serum measurements were obtained for each of n =
442 diabetes patients, as well as the response of interest, a
quantitative measure of disease progression one year after baseline.

**Data Set Characteristics:**

  :Number of Instances: 442

  :Number of Attributes: First 10 columns are numeric predictive values

  :Target: Column 11 is a quantitative measure of disease progression one year after baseline

  :Attribute Information:
      - Age
      - Sex
      - Body mass index
      - Average blood pressure
      - S1
      - S2
      - S3
      - S4
      - S5
      - S6

Note: Each of these 10 feature variables have been mean centered and scaled by the standard deviation times `n_samples` (i.e. the sum of squares of each column totals 1).

Source URL:
https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html

For more information see:
Bra

* We have too much data in this dataset `diabetes_data`, take only the third column of the dataset and store it in a `diabetes_X` variable.

In [142]:
diabetes_X = diabetes.data[:,2]


In [143]:
y = diabetes.data[:,-1]


* Visualize your data using `plotly`.

In [144]:
import plotly.express as px
from IPython.display import HTML, Image
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default ='colab'
fig = px.scatter(x=diabetes_X, y=y, title= 'scatter plot of BMI against diabetes metric')
fig.show()

* Now we need to define a cost function. For a linear regression, we could use MSE :

`np.mean((model(input) - y)**2)`

  * Create this function which we'll call `mse` (for mean square error). This function will take two arguments `y_pred` & `y_true`.

In [145]:
def mse(y_pred, y_true):
  return np.mean((y_pred - y_true)**2)

* Test your function by inserting `model(diabetes_X)` & `y` as arguments. 
* Calculate the rmse as well

In [146]:
print('mse :', mse(model(diabetes_X), y),'\n',
'rmse :', np.sqrt(mse(model(diabetes_X), y)))

mse : 0.007450400491885717 
 rmse : 0.08631570246418503


* Visualize your regression in relation to your points 

In [147]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=diabetes_X,
                         y=y, 
                         name='target',
                         mode='markers'
                         ))
fig.add_trace(go.Scatter(x = diabetes_X ,
                         y = model(diabetes_X),
                         name= 'predictions'
                         ))
fig.update_layout(title= dict(text ="Target vs Prediction"))

fig.show()

* We're going to need to compute the gradients for our variable `model.beta_1` and our constant `model.beta_0`. To do this, we're going to need to review our derivative formulas. Since we're not here to do math, we're going to give you these formulas. 
  * `derive_model_beta_1 = 2/len(y_pred)*np.sum((x @ (y_pred - y_true)))`
  * `derive_model_beta_0 = 2/len(y_pred)*(np.sum(y_pred - y_true))`

  * Feel free to read this article if you want to know more about the calculation of the derivative: [Gradient Descent Derivation](https://mccormickml.com/2014/03/04/gradient-descent-derivation/)


  * So using the above formulas, code the first function `derivative_mse_beta_1` that will take the arguments: 
    * `x` --> the values for your variable / `y_pred` --> the values predicted by your model / `y_true` --> the values of the target variable


In [150]:
def derivative_mse_beta_1(x,y_pred, y_true):
 return 2/len(y_pred)*np.sum((np.matrix(x) @ (y_pred - y_true)))

derivative_mse_beta_1(diabetes_X, model(diabetes_X), y)

-0.00152145875857723

* Test you function

In [151]:
derivative_mse_beta_1(diabetes_X, model(diabetes_X), y)

-0.00152145875857723

* So using the above formulas, now code the `derivative_mse_beta_0` function which will take the arguments :
    * `y_pred` --> the values predicted by your model / `y_true` --> the actual values to predict

In [152]:
def derivative_mse_beta_0(y_pred, y_true):
  return 2/len(y_pred)*(np.sum(y_pred - y_true))


* Test you function

In [153]:
derivative_mse_beta_0(model(diabetes_X), y)

0.14524406711871202

* We will try to see if we can minimize our cost function using the two gradients above. To update our variables, we need to subtract their respective gradients. Ex: 
  * `param = param - learning_rate * gradient`

  * Set a `learning_rate` to 0.1
  * Try to apply your formula on `model.beta_1` and `model.beta_0`.

In [155]:
learning_rate = 0.1
# param = param - learning_rate * gradient

print('OLD model.beta_1 = ', model.beta_1)
print('OLD model.beta_0 = ', model.beta_0)

model.beta_1 = model.beta_1 - learning_rate * derivative_mse_beta_1(diabetes_X, model(diabetes_X), y)

model.beta_0  = model.beta_0   - learning_rate * derivative_mse_beta_0(model(diabetes_X), y)
print('NEW model.beta_1 = ', model.beta_1)
print('NEW model.beta_0 = ', model.beta_0)



OLD model.beta_1 =  [0.05258975]
OLD model.beta_0 =  [0.05809763]
NEW model.beta_1 =  [0.05274183]
NEW model.beta_0 =  [0.0464781]


OLD model.a = [0.38650821]
OLD model.b = [-0.41664214]
NEW model.a = [0.81594204]
NEW model.b = [30.09338312]


Wesee that the values of the two parameters have changed, let's see how it affected the predictions of the model. 
Visualize the data vs the model's predictions.

In [156]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=diabetes_X,
                         y=y, 
                         name='target',
                         mode='markers'
                         ))
fig.add_trace(go.Scatter(x = diabetes_X ,
                         y = model(diabetes_X),
                         name= 'predictions'
                         ))
fig.update_layout(title= dict(text ="Target vs Prediction"))

fig.show()

We notice the predictions got a little closer to our real data
* Recalculate your MSE

In [157]:
print('mse :', mse(model(diabetes_X), y))

mse : 0.004336191997373266


* Our MSE has dropped a lot! This is good news but the process of gradient descent is iterative. So you'll have to do it several times before arriving at accurate predictions. 
  * By making a loop, try to repeat the process from above 10,000 times. 
  * Display every 1000 epochs: mse, model.beta_1 & model.beta_0 

In [158]:
beta_1 = derivative_mse_beta_1(diabetes_X, model(diabetes_X), y)
beta_0 = derivative_mse_beta_0(model(diabetes_X), y)

i = 0
while i < 1000 + 1:
    
  model.beta_1 = model.beta_1 - learning_rate * derivative_mse_beta_1(diabetes_X, model(diabetes_X), y)

  model.beta_0  = model.beta_0   - learning_rate * derivative_mse_beta_0(model(diabetes_X), y)
  i += 1

  if i % 10 == 0:
    print('-------------------- Epoch {}-------------------'.format(i/10))
    print('Current loss : ', mse(model(diabetes_X), y)) 
    print('beta_1 = ', [beta_1])
    print('beta_0 = ', [beta_0])
    print('--------------------------------------------------')


-------------------- Epoch 1.0-------------------
Current loss :  0.002198582907065585
beta_1 =  [-0.0015200821843354322]
beta_0 =  [0.09295620295597568]
--------------------------------------------------
-------------------- Epoch 2.0-------------------
Current loss :  0.002171684465801597
beta_1 =  [-0.0015200821843354322]
beta_0 =  [0.09295620295597568]
--------------------------------------------------
-------------------- Epoch 3.0-------------------
Current loss :  0.0021691411676253375
beta_1 =  [-0.0015200821843354322]
beta_0 =  [0.09295620295597568]
--------------------------------------------------
-------------------- Epoch 4.0-------------------
Current loss :  0.002166898788268148
beta_1 =  [-0.0015200821843354322]
beta_0 =  [0.09295620295597568]
--------------------------------------------------
-------------------- Epoch 5.0-------------------
Current loss :  0.002164679820076536
beta_1 =  [-0.0015200821843354322]
beta_0 =  [0.09295620295597568]
-------------------------

In [159]:
beta_1

-0.0015200821843354322

* Using `plotly`, view your model and actual values again

In [160]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=diabetes_X,
                         y=y, 
                         name='target',
                         mode='markers'
                         ))
fig.add_trace(go.Scatter(x = diabetes_X ,
                         y = model(diabetes_X),
                         name= 'predictions'
                         ))
fig.update_layout(title= dict(text ="Target vs Prediction"))

fig.show()

**We've got a nice regression this time!** 